# Experiment functionality tutorial with the Investing Algorithm Framework

Welcome to the tutorial about the Experiment functionality within the Investing Algorithm Framework. In this guide, we'll demonstrate how to leverage this feature to conduct A/B Testing effectively. With Experiment, you can effortlessly compare the performance of multiple models, such as challenger versus champion, and experiment with diverse backtest date ranges and parameter configurations.

The Experiment functionality proves invaluable for fine-tuning strategy parameters and comparing different strategies, whether it's testing a new challenger model against the existing production model (champion), or exploring variations in configuration. This process of comparing the challenger model to the production model, or champion, falls under the umbrella of model validation or model evaluation. It entails meticulously assessing the challenger model's performance in relation to the established production model.

## Step 1: Importing the primary and challenger algorithms

In [4]:
# Import the factory methods for creation of the champion and challenger strategies
from algorithms import create_primary_algorithm, create_challenger_algorithm

## Step 2: Setting up the backtest

In [7]:
from investing_algorithm_framework import create_app, BacktestDateRange, CCXTOHLCVMarketDataSource, RESOURCE_DIRECTORY, PortfolioConfiguration
from plotly import graph_objects as go
from pandas import DataFrame as PandasDataFrame
from datetime import datetime

# Define the backtest date ranges
up_turn_date_range = BacktestDateRange(
    start_date=datetime(2022, 12, 20),
    end_date=datetime(2023, 6, 1),
    name="up_turn"
)
sideways_date_range = BacktestDateRange(
    start_date=datetime(2022, 6, 10),
    end_date=datetime(2023, 1, 10),
    name="sideways"
)
down_turn_date_range = BacktestDateRange(
    start_date=datetime(2021, 12, 21),
    end_date=datetime(2022, 6, 20),
    name="down_turn"
)

# Create the app and the champion and challenger algorithms
app = create_app(config={RESOURCE_DIRECTORY: "resources_dump"})
app.add_portfolio_configuration(PortfolioConfiguration(initial_balance=1000, trading_symbol="EUR", market="BITVAVO"))

champion = create_primary_algorithm(
    name="primary",
    description="This is the primary algorithm configured with 21 ema and 50 ema",
    long_period=50,
    short_period=21
)
challenger = create_challenger_algorithm(
    name="secondary",
    description="This is the primary algorithm configured with 21 ema and 50 ema",
    long_period=50,
    short_period=21,
    rsi_period=14,
    rsi_sell_threshold=70,
    rsi_buy_threshold=60
)

def create_prices_graph(data: PandasDataFrame, name: str = "Close", color: str = "blue") -> go.Scatter:
    return go.Scatter(
        x=data.index,
        y=data['Close'],
        mode='lines',
        line=dict(color=color, width=1),
        name=name
    )

down_turn_data_source = CCXTOHLCVMarketDataSource(window_size=100, timeframe="2h", market="bitvavo", symbol="BTC/EUR", identifier="BTC/EUR_OHLCV")
df_down_turn = down_turn_data_source.get_data(start_date=down_turn_date_range.start_date, end_date=down_turn_date_range.end_date).to_pandas()
df_down_turn.set_index('Datetime', inplace=True)
up_turn_data_source = CCXTOHLCVMarketDataSource(window_size=100, timeframe="2h", market="bitvavo", symbol="BTC/EUR", identifier="BTC/EUR_OHLCV")
df_up_turn = up_turn_data_source.get_data(start_date=up_turn_date_range.start_date, end_date=up_turn_date_range.end_date).to_pandas()
df_up_turn.set_index('Datetime', inplace=True)
side_way_data_source = CCXTOHLCVMarketDataSource(window_size=100, timeframe="2h", market="bitvavo", symbol="BTC/EUR", identifier="BTC/EUR_OHLCV")
df_side_way = side_way_data_source.get_data(start_date=sideways_date_range.start_date, end_date=sideways_date_range.end_date).to_pandas()
df_side_way.set_index('Datetime', inplace=True)

from plotly.subplots import make_subplots
fig = make_subplots(rows=3, cols=1, shared_xaxes=False, vertical_spacing=0.02)
fig.add_trace(create_prices_graph(df_down_turn, name="Close down turn", color="red"), row=1, col=1)
fig.add_trace(create_prices_graph(df_up_turn, name="Close up turn", color="green"), row=2, col=1)
fig.add_trace(create_prices_graph(df_side_way, name="Close side ways", color="blue"), row=3, col=1)

fig.update_layout(height=600, width=800, title_text="Backtest date range close prices")

# Add titles to each subplot
fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_yaxes(title_text="Prices down turn", row=1, col=1)
fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_yaxes(title_text="Prices up turn", row=2, col=1)
fig.update_xaxes(title_text="Date", row=3, col=1)
fig.update_yaxes(title_text="Prices side way", row=3, col=1)
fig.show()

## Step 3: Running the backtests for all the date ranges

In [4]:
reports = app.run_backtests(algorithms=[champion, challenger], date_ranges=[down_turn_date_range, up_turn_date_range, sideways_date_range])

Running backtests for date range: down_turn 2021-12-21 00:00:00 - 2022-06-20 00:00:00 for a total of 2 algorithms.


Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 1036.78it/s]
Running backtest for algorithm with name secondary: 100%|██████████| 2173/2173 [00:09<00:00, 224.02it/s]


Running backtests for date range: up_turn 2022-12-20 00:00:00 - 2023-06-01 00:00:00 for a total of 2 algorithms.


Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 1174.88it/s]
Running backtest for algorithm with name secondary: 100%|██████████| 1957/1957 [00:07<00:00, 270.07it/s]


Running backtests for date range: sideways 2022-06-10 00:00:00 - 2023-01-10 00:00:00 for a total of 2 algorithms.


Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 1236.71it/s]
Running backtest for algorithm with name secondary: 100%|██████████| 2569/2569 [00:09<00:00, 268.31it/s]


## Step 4: Analyzing the backtest results

In [5]:
from investing_algorithm_framework import pretty_print_backtest_reports_evaluation, BacktestReportsEvaluation

evaluation = BacktestReportsEvaluation(reports)
pretty_print_backtest_reports_evaluation(evaluation)


              :%%%#+-          .=*#%%%      Backtest reports evaluation
              *%%%%%%%+------=*%%%%%%%-     ---------------------------
              *%%%%%%%%%%%%%%%%%%%%%%%-     Number of reports: 6 backtest reports
              .%%%%%%%%%%%%%%%%%%%%%%#      Total number of date ranges:3
               #%%%####%%%%%%%%**#%%%+      Largest overall profit: (Algorithm primary) 121.6183 EUR 12.1618% (up_turn 2022-12-20 00:00:00 - 2023-06-01 00:00:00) 
         .:-+*%%%%- -+..#%%%+.+-  +%%%#*=-: Largest overall growth: (Algorithm primary) 127.5655 EUR 12.7565% (up_turn 2022-12-20 00:00:00 - 2023-06-01 00:00:00)
          .:-=*%%%%. += .%%#  -+.-%%%%=-:.. 
          .:=+#%%%%%*###%%%%#*+#%%%%%%*+-:  
                +%%%%%%%%%%%%%%%%%%%=       
            :++  .=#%%%%%%%%%%%%%*-         
           :++:      :+%%%%%%#-.            
          :++:        .%%%%%#=              
         :++:        .#%%%%%#*=             
        :++-        :%%%%%%%%%+=            
       .++-   

## Step 5: Highlighting the winning algorithm

In [13]:
from investing_algorithm_framework import pretty_print_backtest, create_trade_exit_markers_chart, create_trade_entry_markers_chart
import pandas as pd

# Set the weights for the evaluation criteria, where we prioritize the profit over the growth
algorithm_name = evaluation.rank(weight_profit=0.7, weight_growth=0.3)
print(f"The winning algorithm is {algorithm_name}")

# Highlight the winning strategy by displaying the backtest result of the up turn date range
up_turn_report = evaluation.get_report(name=algorithm_name, backtest_date_range=up_turn_date_range)
pretty_print_backtest(up_turn_report)

from plotly.subplots import make_subplots
fig = make_subplots(rows=1, cols=1, shared_xaxes=False, vertical_spacing=0.02)
df_up_turn["Datetime"] = pd.to_datetime(df_up_turn.index)
df_up_turn.set_index('Datetime', inplace=True)

fig.add_trace(create_prices_graph(df_up_turn, name="Close price up turn", color="blue"), row=1, col=1)
fig.add_trace(create_trade_entry_markers_chart(df_up_turn, up_turn_report.trades), row=1, col=1)
fig.add_trace(create_trade_exit_markers_chart(df_up_turn, up_turn_report.trades), row=1, col=1)
fig.update_layout(height=600, width=800, title_text="Up turn backtest date range trades overview for best performing algorithm")
fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_yaxes(title_text="Prices up turn", row=1, col=1)
fig.show()

The winning algorithm is primary

                  :%%%#+-          .=*#%%%        Backtest report
                  *%%%%%%%+------=*%%%%%%%-       ---------------------------
                  *%%%%%%%%%%%%%%%%%%%%%%%-       Start date: 2022-12-20 00:00:00
                  .%%%%%%%%%%%%%%%%%%%%%%#        End date: 2023-06-01 00:00:00
                   #%%%####%%%%%%%%**#%%%+        Number of days: 163 
             .:-+*%%%%- -+..#%%%+.+-  +%%%#*=-:   Number of runs: 1957
              .:-=*%%%%. += .%%#  -+.-%%%%=-:..   Number of orders: 39
              .:=+#%%%%%*###%%%%#*+#%%%%%%*+-:    Initial balance: 1000.0
                    +%%%%%%%%%%%%%%%%%%%=         Final balance: 1127.5655
                :++  .=#%%%%%%%%%%%%%*-           Total net gain: 121.6183 12.16%
               :++:      :+%%%%%%#-.              Growth: 127.5655 12.76%
              :++:        .%%%%%#=                Number of trades closed: 19
             :++:        .#%%%%%#*=               Number of trad

## Step 6: Optimizing the winning strategy with different parameters

In [12]:
from investing_algorithm_framework import pretty_print_backtest_reports_evaluation, BacktestReportsEvaluation

configurations = [
    {
        "name": "primary_21_50",
        "description": "This is the primary algorithm configured with 21 ema and 50 ema",
        "long_period": 50,
        "short_period": 21
    },
    {
        "name": "primary_21_75",
        "description": "This is the primary algorithm configured with 21 ema and 50 ema",
        "long_period": 75,
        "short_period": 21
    },
    {
        "name": "primary_50_100",
        "description": "This is the primary algorithm configured with 21 ema and 50 ema",
        "long_period": 100,
        "short_period": 50
    },
    {
        "name": "primary_50_200",
        "description": "This is the primary algorithm configured with 21 ema and 50 ema",
        "long_period": 200,
        "short_period": 50
    }
]
algorithms = []

for configuration in configurations:
    algorithm = create_primary_algorithm(**configuration)
    algorithms.append(algorithm)

reports = app.run_backtests(algorithms=algorithms, date_ranges=[down_turn_date_range, up_turn_date_range, sideways_date_range])
evaluation = BacktestReportsEvaluation(reports)
pretty_print_backtest_reports_evaluation(evaluation, backtest_date_range=up_turn_date_range)
winning_configuration = evaluation.rank(weight_profit=0.7, weight_growth=0.3, backtest_date_range=up_turn_date_range)
print(f"The winning configuration for the up turn backtest date range is {winning_configuration}")

pretty_print_backtest_reports_evaluation(evaluation, backtest_date_range=sideways_date_range)
winning_configuration = evaluation.rank(weight_profit=0.7, weight_growth=0.3, backtest_date_range=sideways_date_range)
print(f"The winning configuration for the side ways backtest date range is {winning_configuration}")

pretty_print_backtest_reports_evaluation(evaluation, backtest_date_range=down_turn_date_range)
winning_configuration = evaluation.rank(weight_profit=0.7, weight_growth=0.3, backtest_date_range=down_turn_date_range)
print(f"The winning configuration for the down turn backtest date range is {winning_configuration}")

Running backtests for date range: down_turn 2021-12-21 00:00:00 - 2022-06-20 00:00:00 for a total of 4 algorithms.


Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 875.64it/s]
Running backtest for algorithm with name primary_21_50: 100%|██████████| 2173/2173 [00:12<00:00, 170.01it/s]
Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 1153.55it/s]
Running backtest for algorithm with name primary_21_75: 100%|██████████| 2173/2173 [00:10<00:00, 202.24it/s]
Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 778.24it/s]
Running backtest for algorithm with name primary_50_100: 100%|██████████| 2173/2173 [00:09<00:00, 234.25it/s]
Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 1085.76it/s]
Running backtest for algorithm with name primary_50_200: 100%|██████████| 2173/2173 [00:09<00:00, 231.95it/s]


Running backtests for date range: up_turn 2022-12-20 00:00:00 - 2023-06-01 00:00:00 for a total of 4 algorithms.


Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 978.04it/s]
Running backtest for algorithm with name primary_21_50: 100%|██████████| 1957/1957 [00:09<00:00, 202.39it/s]
Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 1133.75it/s]
Running backtest for algorithm with name primary_21_75: 100%|██████████| 1957/1957 [00:08<00:00, 218.84it/s]
Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 1178.51it/s]
Running backtest for algorithm with name primary_50_100: 100%|██████████| 1957/1957 [00:07<00:00, 249.54it/s]
Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 960.23it/s]
Running backtest for algorithm with name primary_50_200: 100%|██████████| 1957/1957 [00:07<00:00, 264.58it/s]


Running backtests for date range: sideways 2022-06-10 00:00:00 - 2023-01-10 00:00:00 for a total of 4 algorithms.


Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 1031.05it/s]
Running backtest for algorithm with name primary_21_50: 100%|██████████| 2569/2569 [00:12<00:00, 209.66it/s]
Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 1356.06it/s]
Running backtest for algorithm with name primary_21_75: 100%|██████████| 2569/2569 [00:11<00:00, 217.44it/s]
Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 949.69it/s]
Running backtest for algorithm with name primary_50_100: 100%|██████████| 2569/2569 [00:11<00:00, 232.72it/s]
Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 1103.04it/s]
Running backtest for algorithm with name primary_50_200: 100%|██████████| 2569/2569 [00:10<00:00, 240.03it/s]


              :%%%#+-          .=*#%%%      Backtest reports evaluation
              *%%%%%%%+------=*%%%%%%%-     ---------------------------
              *%%%%%%%%%%%%%%%%%%%%%%%-     Number of reports: 4 backtest reports
              .%%%%%%%%%%%%%%%%%%%%%%#      Largest overall profit: (Algorithm primary_21_50) 121.6183 EUR 12.1618% (up_turn 2022-12-20 00:00:00 - 2023-06-01 00:00:00) 
               #%%%####%%%%%%%%**#%%%+      Largest overall growth: (Algorithm primary_21_50) 127.5655 EUR 12.7565% (up_turn 2022-12-20 00:00:00 - 2023-06-01 00:00:00)
         .:-+*%%%%- -+..#%%%+.+-  +%%%#*=-: 
          .:-=*%%%%. += .%%#  -+.-%%%%=-:.. 
          .:=+#%%%%%*###%%%%#*+#%%%%%%*+-:  
                +%%%%%%%%%%%%%%%%%%%=       
            :++  .=#%%%%%%%%%%%%%*-         
           :++:      :+%%%%%%#-.            
          :++:        .%%%%%#=              
         :++:        .#%%%%%#*=             
        :++-        :%%%%%%%%%+=            
       .++-        -%%%%%%%%%%%

## Conclusion

In this guide we have demonstrated how to leverage the backtests functionality within the Investing Algorithm Framework to conduct A/B Testing effectively. We have compared the performance of multiple models, such as challenger versus champion, and experimented with diverse backtest date ranges and parameter configurations. This process of comparing the challenger model to the production model, or champion, falls under the umbrella of model validation or model evaluation. It entails meticulously assessing the challenger model's performance in relation to the established production model. The Experiment functionality proves invaluable for fine-tuning strategy parameters and comparing different strategies, whether it's testing a new challenger model against the existing production model (champion), or exploring variations in configuration.

Also, when selected an algorithm we showed how to test different configurations of the winning algorithm to optimize the strategy. This process is crucial to ensure that the algorithm is performing at its best and to maximize the profit. By following these steps, you can effectively compare different algorithms and configurations to identify the best performing strategy for your trading needs.